In [ ]:
'''
    remote_exec
    IBM/Dell Qiskit Runtime Emulator Demo
    July 26, 2021
    
    
    The following notebook walks through a (simple) example usage of the 
    Qiskit Runtime Emulator in a remote execution environment: i.e. one in
    a Kubernetes cluster or deployed anywhere that is not the client machine.
'''


from qiskit_emulator import DellHybridProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

In [ ]:
'''
    The main difference between the local and remote runtime interfaces
    is here: `provider.remote(URL_OF_REMOTE)`
    
    This function connects the client to the remote environment and, if 
    necessary, logs them in through SSO if enabled. 
'''

provider = DellHybridProvider()
provider.remote(REMOTE_RUNTIME)

In [ ]:
'''
    Uploading a program works exactly the same as locally
'''

text_program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {text_program_id}")

In [ ]:
programs = provider.runtime.pprint_programs(refresh=True)

In [ ]:
provider.runtime.update_program(text_program_id, description="IBM/Dell Updated Qiskit Runtime Program")

In [ ]:
programs = provider.runtime.pprint_programs(refresh=True)

In [ ]:
'''
    This can also be done locally - but if the provided string to upload_program
    is a file or directory path, the upload function will grab the contents 
    and send those to the remote orchestrator.
'''


file_program_id = provider.runtime.upload_program("qka.py", description="File Upload to Orchestrator")

In [ ]:
dir_program_id = provider.runtime.upload_program("./qkad", description="Directory Upload to Orchestrator")

In [ ]:
provider.runtime.pprint_programs(refresh=True)

In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [ ]:
job = provider.runtime.run(text_program_id, options=None, inputs=program_inputs)

In [ ]:
results = job.result(timeout=60)
print(results)

In [ ]:
def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')
    

In [ ]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(dir_program_id, inputs=program_inputs, options=None, callback=callback_function)

In [ ]:
results = job.result(timeout=600)
print(results)